In [1]:
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import os, pickle
os.chdir('/Users/kathleenhamilton/Desktop/QHACK2021/qose/QHACK2021/src')
from subarchitecture_tree_search import *

## First example

The first example builds a circuit classifier for the moons dataset available in sklearn.  The architecture consists of:

`Encoding` -> `Parameterized Layers` -> `One Hot Encoding`

The tree will iteratively grow the parameterized layers through the addition of:
* Layers of RX gates
* Layers of ZZ gates
* Layers of RY gates
* Layers of RZ gates

The layout of the `ZZ` gates connects the qubits in a cycle.  

In [3]:
# Create a unique name for your experiment
EXPERIMENT_NAME = 'OHE_4Q_moons'

# Create a directory to store the data
if not os.path.exists('data'):
    os.mkdir('data/')

data_path = f'data/{EXPERIMENT_NAME}'
if not os.path.exists(data_path):
    os.mkdir(data_path)

# Create a configuration file for the tree prune algorithm
config = {'nqubits': 4,
          'min_tree_depth': 3,
          'max_tree_depth': 9,
          'prune_rate': 0.3,
          'prune_step': 2,
          'plot_trees': False,
          'data_set': 'moons',
          'nsteps': 8,
          'optim': qml.AdamOptimizer,
          'batch_sizes': [8,16,32],
          'n_samples': 1000,
          'learning_rates': [0.1,0.2],
          'save_frequency': 1,
          'save_path': data_path,
          'save_timing': False,
          'circuit_type':'schuld',
          'Tmax': [100,100,100],
          'inf_time':'timeit',
          'fill':'redundant', # or 'pad'
          'rate_type': 'batch_cost', # 'accuracy' or 'batch_cost'
          'readout_layer': 'one_hot',  #'one_hot' or 'weighted_neuron'
          }

# Save the configuration file so that we can remember what we did
with open(data_path + '/config.pickle', 'wb') as f:
    pickle.dump(config, f)
    
# Execute the algorithm
run_tree_architecture_search(config)

In [4]:
# Create a unique name for your experiment
EXPERIMENT_NAME = 'OHE_6Q_moons'

# Create a directory to store the data
if not os.path.exists('data'):
    os.mkdir('data/')

data_path = f'data/{EXPERIMENT_NAME}'
if not os.path.exists(data_path):
    os.mkdir(data_path)

# Create a configuration file for the tree prune algorithm
config = {'nqubits': 6,
          'min_tree_depth': 3,
          'max_tree_depth': 9,
          'prune_rate': 0.3,
          'prune_step': 2,
          'plot_trees': False,
          'data_set': 'moons',
          'nsteps': 7,
          'optim': qml.AdamOptimizer,
          'batch_sizes': [8,16,32],
          'n_samples': 1000,
          'learning_rates': [0.1,0.2],
          'save_frequency': 1,
          'save_path': data_path,
          'save_timing': False,
          'circuit_type':'schuld',
          'Tmax': [100,100,100],
          'inf_time':'timeit',
          'fill':'redundant', # or 'pad'
          'rate_type': 'batch_cost', # 'accuracy' or 'batch_cost'
          'readout_layer': 'one_hot',  #'one_hot' or 'weighted_neuron'
          }

# Save the configuration file so that we can remember what we did
with open(data_path + '/config.pickle', 'wb') as f:
    pickle.dump(config, f)
    
# Execute the algorithm
run_tree_architecture_search(config)

Depth = 1
Depth = 2
Current best architecture:  E1
max W: 1.0
weights: []
Training leaf E1:ZZ
error rate: 0.2971957187651047
error rate: 0.20226635392930162
Training leaf E1:X
error rate: 0.23969481660003172
error rate: 0.25720804224043703
Training leaf E1:Y
error rate: 0.22082123500869813
error rate: 0.25676741415751453
Training leaf E1:Z
error rate: 0.16625424333059752
error rate: 0.6395141046034251
Depth = 3
Prune Tree
Current best architecture:  E1:Z
max W: 5.622061409299822
weights: [[ 3.51726004e-11  2.39675591e-01]
 [-4.14884298e-12  1.46265018e-01]
 [ 3.09494084e-11  1.52481032e-01]
 [ 2.11825018e-11  6.45684414e-01]
 [ 2.07064807e-11  7.02761186e-01]
 [ 2.60037288e-11  1.29577607e-01]]
Grow Pruned Tree
Training leaf E1:ZZ:X
error rate: 0.37139471201616137
error rate: 0.21669741231439868
Training leaf E1:ZZ:Y
error rate: 0.2473286013258816
error rate: 0.2140341852695207
Training leaf E1:ZZ:Z
error rate: 0.19887569670092248
error rate: 0.2080654820554136
Training leaf E1:Z:ZZ
er

error rate: 0.1845471002274641
error rate: 0.4957410844822491
Training leaf E1:Z:Y:ZZ:X:Y
error rate: 0.20862825261733128
error rate: 0.19424063727949484
Training leaf E1:Z:Y:ZZ:X:Z
error rate: 0.23151882727606735
error rate: 0.22335405394701718
Training leaf E1:Z:Y:ZZ:Y:ZZ
error rate: 0.19561353106990606
error rate: 0.18134095331180897
Training leaf E1:Z:Y:ZZ:Y:X
error rate: 0.3255119103433119
error rate: 0.21931403252255435
Training leaf E1:Z:Y:ZZ:Y:Z
error rate: 0.20784782167461194
error rate: 0.2716562520668246
Training leaf E1:Z:Y:ZZ:Z:ZZ
error rate: 0.2601745798694429
error rate: 0.22259817885714667
Training leaf E1:Z:Y:ZZ:Z:X
error rate: 0.2479795778447475
error rate: 0.30206163155990334
Training leaf E1:Z:Y:ZZ:Z:Y
error rate: 0.25651499763955615
error rate: 0.3409820588084644
Training leaf E1:Z:Y:X:ZZ:X
error rate: 0.25517255604693495
error rate: 0.2543354913282918
Training leaf E1:Z:Y:X:ZZ:Y
error rate: 0.21665047179702518
error rate: 0.31985537157720245
Training leaf E1:Z:Y:X

error rate: 0.4044529225867571
error rate: 0.21449417322129036
Training leaf E1:ZZ:Z:ZZ:X:Y
error rate: 0.20854946483844228
error rate: 0.18309378081208133
Training leaf E1:ZZ:Z:ZZ:X:Z
error rate: 0.2101509928288568
error rate: 0.34016500829838087
Training leaf E1:ZZ:Z:ZZ:Y:ZZ
error rate: 0.3094796660116563
error rate: 0.27778698874287805
Training leaf E1:ZZ:Z:ZZ:Y:X
error rate: 0.2646700024102033
error rate: 0.21715720001734037
Training leaf E1:ZZ:Z:ZZ:Y:Z
error rate: 0.18852273969432976
error rate: 0.2129964374377668
Training leaf E1:ZZ:Z:ZZ:Z:ZZ
error rate: 0.21263820676493372
error rate: 0.2009299907576916
Training leaf E1:ZZ:Z:ZZ:Z:X
error rate: 0.2429245036528577
error rate: 0.28677932718039867
Training leaf E1:ZZ:Z:ZZ:Z:Y
error rate: 0.19677713680563347
error rate: 0.2761882246174217
Depth = 7
Prune Tree
Current best architecture:  E1:ZZ:Z:ZZ:Z
max W: 6.376510951394295
weights: [[-2.16138728e-11  1.14949489e-11 -1.20257292e-11  1.13941251e-11
   4.98752858e-01]
 [-2.63133557e-11

error rate: 0.24745291245025358
error rate: 0.29776505558531063
Training leaf E1:Z:X:ZZ:Z:ZZ:Y
error rate: 0.27724377315269333
error rate: 0.3471560376866675
Training leaf E1:Z:X:ZZ:Z:ZZ:Z
error rate: 0.2733558675621754
error rate: 0.22831041710302227
Training leaf E1:ZZ:Y:X:Z:Y:ZZ
error rate: 0.21479230315861528
error rate: 0.46281251340695245
Training leaf E1:ZZ:Y:X:Z:Y:X
error rate: 0.21202559478876834
error rate: 0.20954992326215055
Training leaf E1:ZZ:Y:X:Z:Y:Z
error rate: 0.19298937492800092
error rate: 0.39369318894553335
Training leaf E1:Z:Y:X:ZZ:Y:ZZ
error rate: 0.25932854915564624
error rate: 0.19659201860378378
Training leaf E1:Z:Y:X:ZZ:Y:X
error rate: 0.35489722230377474
error rate: 0.23410446303367913
Training leaf E1:Z:Y:X:ZZ:Y:Z
error rate: 0.2609731713234671
error rate: 0.23693534339249658
Training leaf E1:ZZ:Z:X:Y:X:ZZ
error rate: 0.22266695979737405
error rate: 0.21412949630899225
Training leaf E1:ZZ:Z:X:Y:X:Y
error rate: 0.2828917266037726
error rate: 0.2060409243879

error rate: 0.6164338935947039
Training leaf E1:Z:ZZ:X:Y:ZZ:Y
error rate: 0.25348110763741855
error rate: 0.22188789588189517
Training leaf E1:Z:ZZ:X:Y:ZZ:Z
error rate: 0.24394797936862772
error rate: 0.2686458388232476
Training leaf E1:Z:Y:X:Y:ZZ:X
error rate: 0.21636492561816015
error rate: 0.21373619562501944
Training leaf E1:Z:Y:X:Y:ZZ:Y
error rate: 0.25724582480912067
error rate: 0.2407659943242222
Training leaf E1:Z:Y:X:Y:ZZ:Z
error rate: 0.3890005602630565
error rate: 0.22388743544260556
Training leaf E1:Z:ZZ:Z:Y:Z:ZZ
error rate: 0.26685880282601243
error rate: 0.37847182870253815
Training leaf E1:Z:ZZ:Z:Y:Z:X
error rate: 0.2374205405925132
error rate: 0.19976254371260538
Training leaf E1:Z:ZZ:Z:Y:Z:Y
error rate: 0.18669756109465105
error rate: 0.25887051532098615
Training leaf E1:Z:ZZ:Y:Z:ZZ:X
error rate: 0.22442415709064986
error rate: 0.2469887841652164
Training leaf E1:Z:ZZ:Y:Z:ZZ:Y
error rate: 0.26998376523713175
error rate: 0.2030325891142476
Training leaf E1:Z:ZZ:Y:Z:ZZ:Z

In [ ]:
dev = qml.device("default.qubit.autograd", wires=config['nqubits'])
test,_,_ = construct_circuit_from_leaf('E1:ZZ:Y:ZZ:X', 4, 2, dev,config)

In [ ]:
test()

In [ ]:
tes

In [ ]:
ta = np.zeros((5,1))

In [ ]:
mat = np.hstack((tes,ta))

In [ ]:
mat[:,:-1]